In [23]:
"""
Created 5-18-17
All of the classes for EloPy. The users should only interact with the Implementation class.
@author - Hank Hang Kai Sheehan
"""
import pandas as pd

class Implementation:
    """
    A class that represents an implementation of the Elo Rating System
    """

    def __init__(self, base_rating=1000):
        """
        Runs at initialization of class object.
        @param base_rating - The rating a new player would have
        """
        self.base_rating = base_rating
        self.players = []

    def __getPlayerList(self):
        """
        Returns this implementation's player list.
        @return - the list of all player objects in the implementation.
        """
        return self.players

    def getPlayer(self, name):
        """
        Returns the player in the implementation with the given name.
        @param name - name of the player to return.
        @return - the player with the given name.
        """
        for player in self.players:
            if player.name == name:
                return player
        return None

    def contains(self, name):
        """
        Returns true if this object contains a player with the given name.
        Otherwise returns false.
        @param name - name to check for.
        """
        for player in self.players:
            if player.name == name:
                return True
        return False

    def addPlayer(self, name, rating=None):
        """
        Adds a new player to the implementation.
        @param name - The name to identify a specific player.
        @param rating - The player's rating.
        """
        if rating == None:
            rating = self.base_rating

        self.players.append(_Player(name=name,rating=rating))

    def removePlayer(self, name):
        """
        Adds a new player to the implementation.
        @param name - The name to identify a specific player.
        """
        self.__getPlayerList().remove(self.getPlayer(name))


    def recordMatch(self, name1, name2, winner=None, draw=False):
        """
        Should be called after a game is played.
        @param name1 - name of the first player.
        @param name2 - name of the second player.
        """
        player1 = self.getPlayer(name1)
        player2 = self.getPlayer(name2)
        
        player1.games = player1.games + 1
        player2.games = player2.games + 1

        expected1 = player1.compareRating(player2)
        expected2 = player2.compareRating(player1)
        
        k = 32 # Change this. Higher k -> leads to more variance in the ratings and more emphasis on last few games

        rating1 = player1.rating
        rating2 = player2.rating

        if draw:
            score1 = 0.5
            score2 = 0.5
            player1.draws = player1.draws + 1
            player2.draws = player2.draws + 1
        elif winner == name1:
            score1 = 1.0
            score2 = 0.0
            player1.wins = player1.wins + 1
            player2.losses = player2.losses + 1
        elif winner == name2:
            score1 = 0.0
            score2 = 1.0
            player1.losses = player1.losses + 1
            player2.wins = player2.wins + 1
        else:
            raise InputError('One of the names must be the winner or draw must be True')

        newRating1 = rating1 + k * (score1 - expected1)
        newRating2 = rating2 + k * (score2 - expected2)

        if newRating1 < 0:
            newRating1 = 0
            newRating2 = rating2 - rating1

        elif newRating2 < 0:
            newRating2 = 0
            newRating1 = rating1 - rating2

        player1.rating = newRating1
        player2.rating = newRating2

    def getPlayerRating(self, name):
        """
        Returns the rating of the player with the given name.
        @param name - name of the player.
        @return - the rating of the player with the given name.
        """
        player = self.getPlayer(name)
        return player.rating

    def getRatingList(self):
        """
        Returns a list of tuples in the form of ({name},{rating})
        @return - the list of tuples
        """
        lst = []
        for player in self.__getPlayerList():
            lst.append((player.name, round(player.rating, 2), player.games, player.wins, player.losses, player.draws))
        return lst

class _Player:
    """
    A class to represent a player in the Elo Rating System
    """

    def __init__(self, name, rating, games = 0, wins = 0, losses = 0, draws = 0):
        """
        Runs at initialization of class object.
        @param name - TODO
        @param rating - TODO
        """
        self.name = name
        self.rating = rating
        self.games = games
        self.wins = wins
        self.losses = losses
        self.draws = draws

    def compareRating(self, opponent):
        """
        Compares the two ratings of the this player and the opponent.
        @param opponent - the player to compare against.
        @returns - The expected score between the two players.
        """
        return (1+10**((opponent.rating-self.rating )/400.0))**-1

In [24]:
games = pd.read_excel('1C Chess.xlsx')

chess = Implementation()

for index, row in games.iterrows():
    white = row['White']
    black = row['Black']
    winner = row['Winner']
    if not chess.contains(white):
        chess.addPlayer(white)
    if not chess.contains(black):
        chess.addPlayer(black)
    if winner == "Draw":
        chess.recordMatch(white, black, draw=True)
    else:
        chess.recordMatch(white, black, winner=winner)

chess.getRatingList()

[('Zack', 1015.32, 18, 10, 7, 1),
 ('Steve', 995.88, 25, 10, 14, 1),
 ('Mitch', 1057.99, 12, 9, 3, 0),
 ('Randall', 947.81, 8, 2, 6, 0),
 ('Pete', 983.0, 5, 2, 3, 0)]